In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

D:\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
D:\Anaconda\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start

In [39]:
# Label mapping
label_map = {
    0: "Negative",
    1: "Neutral",
    2: "Positive"
}

def classify_roberta_sentiment(text):
    # Tokenize the input
    encoded_input = tokenizer(text, return_tensors='pt')
    
    with torch.no_grad():
        logits = model(**encoded_input).logits

    probs = F.softmax(logits, dim=1)[0]

    # Apply custom neutral threshold
    neutral_score = probs[1].item()
    negative_score = probs[0].item()
    positive_score = probs[2].item()
    if neutral_score > 0.3 or (negative_score < 0.49 and positive_score < 0.49):
        sentiment = "Neutral"
    else:
        # Pick between positive and negative
        top_label = torch.argmax(probs).item()
        sentiment = label_map[top_label] if top_label != 1 else "Neutral"  # Safety check

#     return {
#         "sentiment": sentiment,
#         "all_scores": {
#             "Negative": round(probs[0].item(), 3),
#             "Neutral": round(probs[1].item(), 3),
#             "Positive": round(probs[2].item(), 3)
#         }
#     }

    return sentiment

print(classify_roberta_sentiment("I like this shirt"))
print(classify_roberta_sentiment("It was ok"))
print(classify_roberta_sentiment("This is terrible."))
print(classify_roberta_sentiment("Delivery was late, but clothes were comfortable"))


Positive
Positive
Negative
Neutral


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/analyze', methods=['POST'])
def analyze_sentiment():
    # Get the message from the incoming request
    data = request.get_json()
    message = data.get("message", "")
    if not message:
        return jsonify({"error": "No 'message' provided."}), 400

    # Call the sentiment analysis function
    sentiment = classify_roberta_sentiment(message)
    
    # Return the sentiment as a response
    return jsonify({"sentiment": sentiment})

if __name__ == "__main__":
    app.run(port=1212)  


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:1212
Press CTRL+C to quit
127.0.0.1 - - [16/May/2025 16:58:53] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2025 16:59:08] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2025 16:59:36] "POST /analyze HTTP/1.1" 200 -
